# User Guide Tutorial 06: Treatment Effects

This tutorial shows how to use TemporAI `treatments` plugins.

*Uncomment the below to install the library:*

In [1]:
# !pip install temporai

# Or from the repo, for the latest version:
# !pip install git+https://github.com/vanderschaarlab/temporai.git

## All `treatments` plugins

> ⚠️ The `treatments` API is preliminary and likely to change.

In the treatment effects estimation task, the goal is to predict a counterfactual outcome given an alternative treatment.

To see all the relevant plugins:

In [2]:
from tempor.plugins import plugin_loader
from rich.pretty import pprint

all_treatments_plugins = plugin_loader.list()["treatments"]

pprint(all_treatments_plugins, indent_guides=False)

{
    'one_off': {'regression': ['synctwin_regressor']},
    'temporal': {'classification': ['crn_classifier'], 'regression': ['crn_regressor']}
}

## Using a temporal treatment effects plugin.

In this setting, the treatments are time series, and the outcomes are also time series.

In [3]:
from tempor.utils.dataloaders import DummyTemporalTreatmentEffectsDataLoader
from tempor.plugins import plugin_loader

dataset = DummyTemporalTreatmentEffectsDataLoader(
    random_state=42,
    temporal_covariates_missing_prob=0.0,
    temporal_treatments_n_features=1,
    temporal_treatments_n_categories=2,
).load()
print(dataset)

model = plugin_loader.get("treatments.temporal.regression.crn_regressor", epochs=20)
print(model)

TemporalTreatmentEffectsDataset(
    time_series=TimeSeriesSamples([100, *, 5]),
    static=StaticSamples([100, 3]),
    predictive=TemporalTreatmentEffectsTaskData(
        targets=TimeSeriesSamples([100, *, 2]),
        treatments=TimeSeriesSamples([100, *, 1])
    )
)
CRNTreatmentsRegressor(
    name='crn_regressor',
    category='treatments.temporal.regression',
    params={
        'encoder_rnn_type': 'LSTM',
        'encoder_hidden_size': 100,
        'encoder_num_layers': 1,
        'encoder_bias': True,
        'encoder_dropout': 0.0,
        'encoder_bidirectional': False,
        'encoder_nonlinearity': None,
        'encoder_proj_size': None,
        'decoder_rnn_type': 'LSTM',
        'decoder_hidden_size': 100,
        'decoder_num_layers': 1,
        'decoder_bias': True,
        'decoder_dropout': 0.0,
        'decoder_bidirectional': False,
        'decoder_nonlinearity': None,
        'decoder_proj_size': None,
        'adapter_hidden_dims': [50],
        'adapter_out_

In [4]:
# Targets:
dataset.predictive.targets

TimeSeriesSamples with data:
                             0         1
sample_idx time_idx                     
0          0         -3.110475 -3.566948
           1          1.528495 -0.653673
           2          2.275307 -0.695371
           3          4.844060  3.469371
           4          4.420301  5.147500
...                        ...       ...
99         7          5.994185  6.225290
           8         10.913662  5.346697
           9          9.558824  7.585175
           10        10.194430  5.795619
           11        13.774189  8.457336

[1573 rows x 2 columns]

In [5]:
# Treatments:
dataset.predictive.treatments

TimeSeriesSamples with data:
                     0
sample_idx time_idx   
0          0         0
           1         1
           2         1
           3         0
           4         0
...                 ..
99         7         1
           8         1
           9         0
           10        0
           11        1

[1573 rows x 1 columns]

In [6]:
# Train.
model.fit(dataset);

Preparing data for decoder training...
Preparing data for decoder training DONE.
=== Training stage: 1. Train encoder ===
Epoch: 0, Prediction Loss: 76.050, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 76.050
Epoch: 1, Prediction Loss: 37.508, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 37.508
Epoch: 2, Prediction Loss: 21.953, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 21.953
Epoch: 3, Prediction Loss: 18.895, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 18.895
Epoch: 4, Prediction Loss: 19.568, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 19.568
Epoch: 5, Prediction Loss: 16.534, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 16.534
Epoch: 6, Prediction Loss: 12.039, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 12.039
Epoch: 7, Prediction Loss: 10.009, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 10.009
Epoch: 8, Prediction Loss: 8.666, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 8.666
Epoch: 9, Prediction Loss: 6.788, Lambda: 1.000, Treatment BR Loss: 0.000, Loss

In [7]:
# Predict counterfactuals:

import numpy as np

dataset = dataset[:5]

# Define horizons for each sample.
horizons = [tc.time_indexes()[0][len(tc.time_indexes()[0]) // 2 :] for tc in dataset.time_series]
print("Horizons for sample 0:\n", horizons[0], end="\n\n")

# Define treatment scenarios for each sample.
treatment_scenarios = [[np.asarray([1] * len(h)), np.asarray([0] * len(h))] for h in horizons]
print("Alternative treatment scenarios for sample 0:\n", treatment_scenarios[0], end="\n\n")

# Call predict_counterfactuals.
counterfactuals = model.predict_counterfactuals(dataset, horizons=horizons, treatment_scenarios=treatment_scenarios)
print("Counterfactual outcomes for sample 0, given the alternative treatment scenarios:\n")
for idx, c in enumerate(counterfactuals[0]):
    print(f"Treatment scenario {idx}, {treatment_scenarios[0][idx]}")
    print(c, end="\n\n")

Horizons for sample 0:
 [5, 6, 7, 8, 9, 10]

Alternative treatment scenarios for sample 0:
 [array([1, 1, 1, 1, 1, 1]), array([0, 0, 0, 0, 0, 0])]

Counterfactual outcomes for sample 0, given the alternative treatment scenarios:

Treatment scenario 0, [1 1 1 1 1 1]
TimeSeries() with data:
                 0         1
time_idx                    
5         6.339921  5.118720
6         6.379971  5.159422
7         6.380514  5.159956
8         6.380521  5.159964
9         6.380521  5.159964
10        6.380521  5.159964

Treatment scenario 1, [0 0 0 0 0 0]
TimeSeries() with data:
                 0         1
time_idx                    
5         6.694273  5.142348
6         6.732183  5.177425
7         6.732780  5.178019
8         6.732789  5.178029
9         6.732790  5.178029
10        6.732790  5.178029

